In [67]:
from ipyleaflet import Map, basemaps, CircleMarker
from ipywidgets import HTML
import pandas as pd



from ipyleaflet import Icon, Marker, MarkerCluster, Popup

# First is latitude and second is longitude; both in degrees


m = Map(interpolation='nearest', basemap=basemaps.Stamen.Terrain,world_copy_jump=True)

markers = ()


popup = Popup(
            name='popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
def mouseover_callback(marker,location, html):
    def callback(*args, **kwargs):
        popup.child = html
        popup.location = location
        m.add_layer(popup)
    return callback

def mouseout_callback():
    def callback(*args, **kwargs):
        m.remove_layer(popup)
    return callback

from data.labeled import LABELED_CITIES as cities, RISKS_MAPPING as risks
import numpy as np

def populate_per_city(city, country, risks_dict):
    html =  f"""
    <p> <h4><b> {city}, {country} </b></h4>
    
    """
    
    for risk in risks:
        if risk not in risks_dict:
            continue
        risk_value = risks_dict[risk]
        if not pd.isnull(risk_value):
            html += f"""
      <h4><b>{risks[risk]}</b>:{risk_value}</h4>
   
    """
    html += '</p>'
    return HTML(html)
    

for _,city in cities.iterrows():
    location=(city['latitude'],city['longitude'])
    html = populate_per_city(city.city, city.country, city)
    marker = CircleMarker(location=location,fill_color = "blue",color='blue',radius=2)
    marker.on_mouseover(mouseover_callback(marker,location, html))
    marker.on_mouseout(mouseout_callback())
    markers = markers + (marker,)
    
def get_city(latitude, longitude):
    return "TODO"

def get_country(latitude, longitude):
    return "TODO"

def get_risks(latitude, longitude):
    return {}
from math import sqrt
def is_close(p1, p2, thres=1):
    if sqrt((p1[0] - p2[0])**2 + (p1[1]-p2[1])**2) < thres:
        return True
    return False

manually_added_markers = []
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        html = populate_per_city("Close To:" + get_city(coords[0], coords[1]), get_country(coords[0],coords[1]),get_risks(coords[0], coords[1]))
        marker = CircleMarker(location=coords,fill_color = "red",color='red',radius=2)
        check = [is_close(marker.location,mark.location) for mark in manually_added_markers]
        if not any(check):
            marker.on_mouseover(mouseover_callback(marker,coords, html))
            marker.on_mouseout(mouseout_callback())
            manually_added_markers.append(marker)
            m.add_layer(marker)
        else:
            to_rem = [cnt for cnt,(m,c) in enumerate(zip(manually_added_markers,check)) if c][0]
            m.remove_layer(manually_added_markers[to_rem])
            manually_added_markers.pop(to_rem)
            m.remove_layer(popup)

m.on_interaction(handle_click)
m.add_layer(MarkerCluster(markers = markers))

In [68]:
m.layout.width = '80%'
m.layout.height = '1000px'
m.zoom=2.2
m.center = (0, 0)
display(m)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…